지금까진 에이전트 2개로 대화를 이어나갔지만, 더 많은 에이전트끼리 상호작용할 수 있음

즉, 대화 패턴에는 여러가지가 있다!

## Convesation Pattern

- **Two-agent chat:**  
  두 에이전트가 서로 대화를 나누는 가장 단순한 형태의 대화 패턴.

- **Sequential chat:**  
  두 에이전트 간의 연속된 채팅으로, 이전 채팅의 요약이 다음 채팅의 Context로 전달되는 carryover 메커니즘을 사용.

- **Group Chat:**  
  두 명 이상의 에이전트가 참여하는 단일 채팅. 
  그룹 채팅에서는 다음에 어떤 에이전트가 와야하는지가 중요. 그래서 그걸 선택하는 여러 방법이 있음.

  - **다음 에이전트 선택 전략:**  
    `round_robin`, `random`, `manual` (인간 선택), `auto` (기본값, LLM을 사용하여 결정)

  - **다음 에이전트 선택 제약:**  
    다음 에이전트를 제한할 수 있는 방법을 제공.

  - **사용자 정의 선택 함수:**  
    다음 에이전트를 사용자 정의하는 함수를 전달할 수 있음.

- **중첩 채팅 (Nested Chat):**  
  하나의 워크플로우를 단일 에이전트로 패키징하여 더 큰 워크플로우에서 재사용할 수 있음. 2차원 리스트를 생각해보십시오.

## Two-Agent Chat and Chat Result

가장 간단한 형태의 대화 패턴 

각 `ConversableAgent`의 `initiate_chat` 메서드를 사용하여 두 에이전트 간의 채팅을 시작.


![alt text](image/image-3.png)

**Two-Agent Chat **

Two-Agent Chat은 두 가지 입력을 받습니다:  
- **message:** init이 제공하는 문자열  
- **context:** 채팅의 다양한 매개변수를 지정

발신자 에이전트는 자신의 채팅 init 메서드(즉, `ConversableAgent`의 `generate_init_message` 메서드)를 사용하여 입력으로부터 초기 메시지를 생성하고, 이를 수신자 에이전트에게 보내 채팅을 시작. 

여기서 발신자 에이전트는 `initiate_chat` 메서드가 호출된 에이전트이며, 수신자 에이전트는 다른 에이전트.

채팅이 종료되면, 채팅 기록은 채팅 요약기에 의해 처리. 

요약기는 채팅 기록을 요약하고 채팅의 토큰 사용량을 계산함. 

요약 방식은 `initiate_chat` 메서드의 `summary_method` 매개변수를 사용하여 구성할 수 있으며, 기본값은 채팅의 마지막 메시지(`summary_method='last_msg'`)지만, LLM 기반 요약도 있음 (`summary_method="reflection_with_llm"`)


In [1]:
import os

from autogen import ConversableAgent

student_agent = ConversableAgent(
    name="Student_Agent",
    system_message="You are a student willing to learn.",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]},
)
teacher_agent = ConversableAgent(
    name="Teacher_Agent",
    system_message="You are a math teacher.",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

chat_result = student_agent.initiate_chat(
    teacher_agent,
    message="What is triangle inequality?",
    summary_method="reflection_with_llm",
    max_turns=2,
)

Student_Agent (to Teacher_Agent):

What is triangle inequality?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Teacher_Agent (to Student_Agent):

The triangle inequality states that the sum of the lengths of any two sides of a triangle must be greater than the length of the third side. In other words, if a triangle has side lengths of a, b, and c, then the following inequalities must hold:

a + b > c
a + c > b
b + c > a

This concept is important in geometry and helps us determine whether a set of three line segments can form a triangle.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Student_Agent (to Teacher_Agent):

Thank you for explaining the triangle inequality concept. I appreciate your help in understanding this geometry principle.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Teacher_A

In [2]:
print(chat_result.summary)

The triangle inequality states that the sum of the lengths of any two sides of a triangle must be greater than the length of the third side. This principle helps determine whether a set of three line segments can form a triangle.


In [3]:
print(ConversableAgent.DEFAULT_SUMMARY_PROMPT)

Summarize the takeaway from the conversation. Do not add any introductory phrases.


In [4]:
# Get the chat history.
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': 'What is triangle inequality?',
  'name': 'Student_Agent',
  'role': 'assistant'},
 {'content': 'The triangle inequality states that the sum of the lengths of '
             'any two sides of a triangle must be greater than the length of '
             'the third side. In other words, if a triangle has side lengths '
             'of a, b, and c, then the following inequalities must hold:\n'
             '\n'
             'a + b > c\n'
             'a + c > b\n'
             'b + c > a\n'
             '\n'
             'This concept is important in geometry and helps us determine '
             'whether a set of three line segments can form a triangle.',
  'name': 'Teacher_Agent',
  'role': 'user'},
 {'content': 'Thank you for explaining the triangle inequality concept. I '
             'appreciate your help in understanding this geometry principle.',
  'name': 'Student_Agent',
  'role': 'assistant'},
 {'content': "You're welcome! I'm glad I could help you understand the c

In [5]:
# Get the cost of the chat.
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 209,
                                                             'cost': 0.0005785,
                                                             'prompt_tokens': 530,
                                                             'total_tokens': 739},
                                      'total_cost': 0.0005785},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 209,
                                                             'cost': 0.0005785,
                                                             'prompt_tokens': 530,
                                                             'total_tokens': 739},
                                      'total_cost': 0.0005785}}


## Sequential Chat

두 에이전트 간의 연속된 채팅을 의미. 

아래 코드는 이전 채팅의 요약이 다음 채팅의 Context로 전달되는 **carryover** 메커니즘을 사용.

이 패턴은 서로 의존적인 하위 작업들로 나눌 수 있는 복잡한 작업에 특히 유용함. 

1. 두 에이전트가 먼저 Two Agent Chat을 시작
2. 채팅이 종료되면, 대화의 요약이 생성되어 **carryover**로 사용됨.
3. 다음 채팅에서는 이 carryover를 `context`의 carryover 파라미터에 전달하여 초기 메시지를 생성
4. 대화가 진행될수록 carryover가 누적되어, 이후 채팅은 이전 모든 채팅의 요약을 포함하게 됨

![alt text](image/image-4.png)

참고로, 위 그림에서는 각 채팅마다 다른 수신자 에이전트를 표시했지만, 실제로는 수신자 에이전트가 반복될 수 있음.


In [6]:
# The Number Agent always returns the same numbers.
number_agent = ConversableAgent(
    name="Number_Agent",
    system_message="You return me the numbers I give you, one number each line.",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Adder Agent adds 1 to each number it receives.
adder_agent = ConversableAgent(
    name="Adder_Agent",
    system_message="You add 1 to each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Multiplier Agent multiplies each number it receives by 2.
multiplier_agent = ConversableAgent(
    name="Multiplier_Agent",
    system_message="You multiply each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Subtracter Agent subtracts 1 from each number it receives.
subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    system_message="You subtract 1 from each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Divider Agent divides each number it receives by 2.
divider_agent = ConversableAgent(
    name="Divider_Agent",
    system_message="You divide each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

In [7]:
# Start a sequence of two-agent chats.
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat method.
chat_results = number_agent.initiate_chats(
    [
        {
            "recipient": adder_agent,
            "message": "14",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": multiplier_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": subtracter_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": divider_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
Number_Agent (to Adder_Agent):

14

--------------------------------------------------------------------------------
Adder_Agent (to Number_Agent):

15

--------------------------------------------------------------------------------
Number_Agent (to Adder_Agent):

15

--------------------------------------------------------------------------------
Adder_Agent (to Number_Agent):

16

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Number_Agent (to Multiplier_Agent):

These are my numbers
Context: 
16

--------------------------------------------------------------------------------
Mul

In [8]:
print("First Chat Summary: ", chat_results[0].summary)
print("Second Chat Summary: ", chat_results[1].summary)
print("Third Chat Summary: ", chat_results[2].summary)
print("Fourth Chat Summary: ", chat_results[3].summary)

First Chat Summary:  16
Second Chat Summary:  64
Third Chat Summary:  14
62
Fourth Chat Summary:  4  
16  
3.5  
15.5


## Group Chat

지금까지 본 건 결국 다 Two Agent Chat기반이었음.

이젠 진짜 여러 명의 에이전트가 대화하는 것을 볼 수 있다!!

Group Chat의 핵심 아이디어는 모든 에이전트가 대화에 참여하고, 동일한 Context를 공유한다는 점임. 

즉 같은 문맥을 공유하면서 여러 에이전트가 협업을 할 수 있음.

![alt text](image/image-5.png)

**group_chat**

Group Chat은 특수 에이전트 타입인 **GroupChatManager**로 구현 가능. 

Group Chat의 첫 번째 단계에서 Group Chat Manager는 발언할 에이전트를 선택합니다. 선택된 에이전트가 발언하면, 그 메시지가 다시 Group Chat Manager로 전달되고, Manager는 해당 메시지를 그룹 내의 모든 다른 에이전트에게 브로드캐스트함. 이 과정은 대화가 종료될 때까지 반복.

Group Chat Manager는 다음 에이전트를 선택하기 위해 여러 전략을 사용할 수 있는데 다음과 같음.

- **round_robin:**  
  제공된 에이전트 순서에 따라 순환 방식으로 에이전트를 선택.

- **random:**  
  에이전트를 무작위로 선택.

- **manual:**  
  Human Input을 받아 에이전트를 선택.

- **auto:**  
  기본 전략으로, Group Chat Manager의 LLM을 사용하여 에이전트를 선택. LLM이 부르고 싶은 에이전트 선택!


In [9]:
# The `description` attribute is a string that describes the agent.
# It can also be set in `ConversableAgent` constructor.
adder_agent.description = "Add 1 to each input number."
multiplier_agent.description = "Multiply each input number by 2."
subtracter_agent.description = "Subtract 1 from each input number."
divider_agent.description = "Divide each input number by 2."
number_agent.description = "Return the numbers given."

In [10]:
from autogen import GroupChat

group_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    messages=[],
    max_round=6,
)

In [17]:
from autogen import GroupChatManager

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

In [18]:
chat_result = number_agent.initiate_chat(
    group_chat_manager,
    message="My number is 3, I want to turn it into 13.",
    summary_method="reflection_with_llm",
)

Number_Agent (to chat_manager):

My number is 3, I want to turn it into 13.

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

4

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):



--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

13

--------------------------------------------------------------------------------

Next speaker: Number_Agent

Number_Agent (to chat_manager):

3  
4  
13

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

6  
8  
26

--------------------------------------------------------------------------------


In [19]:
print(chat_result.summary)

The user wants to transform the number 3 into 13. The proposed solution involves adding 4 to 3, resulting in 13. Additionally, a multiplication sequence involving unrelated operations is presented without addressing the initial request directly.


# Send Introductions

전에 introduction을 설정한 것은, 다음 에이전트를 LLM이 잘 선택할 수 있도록 한 것이었음.

그러나 이건, Group Chat Manager에게만 도움을 주는 것이고, 개별 에이전트는 해당 안 됨.

그렇지만, 개별 에이전트들도 서로가 뭐하는 에이전트인지 알면 좋다고 함.

그래서 Introduction을 공유할 수 있는데, `send_introductions=True`를 설정하면 됨.




In [20]:
group_chat_with_introductions = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    messages=[],
    max_round=6,
    send_introductions=True,
)

아래는 Group Chat을 Sequential로 하는 것임

In [25]:
# Let's use the group chat with introduction messages created above.
group_chat_manager_with_intros = GroupChatManager(
    groupchat=group_chat_with_introductions,
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

# Start a sequence of two-agent chats between the number agent and
# the group chat manager.
chat_result = number_agent.initiate_chats(
    [
        {
            "recipient": group_chat_manager_with_intros,
            "message": "My number is 3, I want to turn it into 13.",
        },
        {
            "recipient": group_chat_manager_with_intros,
            "message": "Turn this number to 32.",
        },
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
Number_Agent (to chat_manager):

My number is 3, I want to turn it into 13.

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent



/root/miniconda3/envs/autogen/lib/python3.12/site-packages/autogen/agentchat/chat.py:55: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Multiplier_Agent (to chat_manager):

6

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

7

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

Sure, here are the new numbers:

6  
7

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

Thank you for the input. Here are the new numbers after adding 1:

7  
8

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

You're welcome! If you have any more numbers you'd like to transform, feel free to let us know.

--------------------------------------------------------------------------------

********************************************************************************

그런데 이제 `auto`로 돌려두면, 비효율적으로 에이전트를 선택할 수도 있다.

사실 어떤 전략을 선택하든지, 그런 비효율이 발생할 수 있는데

이를 미리 사용자 정의로 제약을 둠으로써 해결할 수 있다!

예를 들어, 소통 가능한 에이전트 팀을 묶는다고 생각하면 편하다~

In [26]:
allowed_transitions = {
    number_agent: [adder_agent, number_agent],
    adder_agent: [multiplier_agent, number_agent],
    subtracter_agent: [divider_agent, number_agent],
    multiplier_agent: [subtracter_agent, number_agent],
    divider_agent: [adder_agent, number_agent],
}

In [27]:
constrained_graph_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    allowed_or_disallowed_speaker_transitions=allowed_transitions,
    speaker_transitions_type="allowed",
    messages=[],
    max_round=12,
    send_introductions=True,
)

constrained_group_chat_manager = GroupChatManager(
    groupchat=constrained_graph_chat,
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

chat_result = number_agent.initiate_chat(
    constrained_group_chat_manager,
    message="My number is 3, I want to turn it into 10. Once I get to 10, keep it there.",
    summary_method="reflection_with_llm",
)

Number_Agent (to chat_manager):

My number is 3, I want to turn it into 10. Once I get to 10, keep it there.

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

4

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

8

--------------------------------------------------------------------------------

Next speaker: Subtracter_Agent

Subtracter_Agent (to chat_manager):

7

--------------------------------------------------------------------------------

Next speaker: Divider_Agent

Divider_Agent (to chat_manager):

3.5

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

The numbers are:

4
8
7
3.5

--------------------------------------------------------------------------------

Next speaker: Number_Agent

Number_A

## nested_chat

Group Chat과 굉장히 비슷해보일 수 있는데 다른 것임.

Group Chat: 여러 에이전트가 협업해서 하나의 문제를 해결
Nested Chat: 단일 에이전트가 자신의 업무를 수행해서 다른 에이전트에게 넘기면 또 자기 업무 수행 (Sequential)

즉, 복잡한 워크플로우가 있는데, 세부적인 단계로 쪼갤 수 있다면 Nested Chat을 사용하면 됨.

![alt text](image/image-6.png)

각 에이전트 단계에서 조건이 충족되면 Sequential Chat 방식을 사용해서 다음 에이전트한테 넘김.

그래서 복잡한 워크플로우를 세부 단계로 나누고, 각 단계에 단일 에이전트를 배치함으로써 문제를 해결할 수 있음.


In [29]:
import tempfile

temp_dir = tempfile.gettempdir()

arithmetic_agent = ConversableAgent(
    name="Arithmetic_Agent",
    llm_config=False,
    human_input_mode="ALWAYS",
    # This agent will always require human input to make sure the code is
    # safe to execute.
    code_execution_config={"use_docker": False, "work_dir": temp_dir},
)

code_writer_agent = ConversableAgent(
    name="Code_Writer_Agent",
    system_message="You are a code writer. You write Python script in Markdown code blocks.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

poetry_agent = ConversableAgent(
    name="Poetry_Agent",
    system_message="You are an AI poet.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

In [30]:
nested_chats = [
    {
        "recipient": group_chat_manager_with_intros,
        "summary_method": "reflection_with_llm",
        "summary_prompt": "Summarize the sequence of operations used to turn " "the source number into target number.",
    },
    {
        "recipient": code_writer_agent,
        "message": "Write a Python script to verify the arithmetic operations is correct.",
        "summary_method": "reflection_with_llm",
    },
    {
        "recipient": poetry_agent,
        "message": "Write a poem about it.",
        "max_turns": 1,
        "summary_method": "last_msg",
    },
]

In [31]:
arithmetic_agent.register_nested_chats(
    nested_chats,
    # The trigger function is used to determine if the agent should start the nested chat
    # given the sender agent.
    # In this case, the arithmetic agent will not start the nested chats if the sender is
    # from the nested chats' recipient to avoid recursive calls.
    trigger=lambda sender: sender not in [group_chat_manager_with_intros, code_writer_agent, poetry_agent],
)

In [32]:
# Instead of using `initiate_chat` method to start another conversation,
# we can use the `generate_reply` method to get single reply to a message directly.
reply = arithmetic_agent.generate_reply(
    messages=[{"role": "user", "content": "I have a number 3 and I want to turn it into 7."}]
)


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

********************************************************************************
Starting a new chat....

********************************************************************************
Arithmetic_Agent (to chat_manager):

I have a number 3 and I want to turn it into 7.

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

4

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

Multiplier_Agent  
6

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

7

--------------------------------------------------------------------------------

Next speaker: Number_Agent

Number_Agent (to chat_manager):

One number per line:  
4  
6  
7  

---------------------